<a href="https://colab.research.google.com/github/FLjv77/Quantum_ML_Course/blob/main/codes/Catalyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Installations

In [1]:
pip install pennylane-catalyst

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.7 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.5.1
    Uninstalling jaxlib-0.5.1:
      Successfully uninstalled jaxlib-0.5.1
  Attempting uninstall: jax
    Found existing installation: jax 

In [2]:
!pip install pennylane

###Imports

In [6]:
from catalyst import qjit, measure, cond, for_loop, while_loop, grad
import pennylane as qml
from jax import numpy as jnp

###Examples

In [4]:
@qml.qnode(qml.device("lightning.qubit", wires=2))
def circuit(theta):
    qml.Hadamard(wires=0)
    qml.RX(theta, wires=1)
    qml.CNOT(wires=[0,1])
    return qml.expval(qml.PauliZ(wires=1))

In [7]:
jitted_circuit = qjit(circuit)
jitted_circuit(0.7)

Array(0., dtype=float64)

In [8]:
@qjit
@qml.qnode(qml.device("lightning.qubit", wires=5))
def circuit(arg0, arg1, arg2):
    qml.RX(arg0, wires=[arg1 + 1])
    qml.RY(arg0, wires=[arg2])
    qml.CNOT(wires=[arg1, arg2])
    return qml.probs(wires=[arg1 + 1])

In [9]:
circuit(jnp.pi / 3, 1, 2)

Array([0.625, 0.375], dtype=float64)

In [16]:
dev = qml.device("lightning.qubit", wires=2)  # any Catalyst-supported backend

@qjit                                       #   ⇦ compile once, run fast
@qml.qnode(dev)
def mid_measure(theta: float):
    qml.Hadamard(0)                         # put qubit-0 in |+⟩

    bit = measure(0)                        # **mid-circuit measurement**
    qml.RX(bit * jnp.pi, wires=1)           # rotate qubit-1 iff bit == 1

    return qml.expval(qml.PauliZ(1)), bit   # return both a quantum & classical value

# --- call the compiled function -----------------------------------------

for i in range(10):
  expectation, outcome = mid_measure(0.0)      # first call triggers compilation
  print(expectation, outcome)


-1.0 True
1.0 False
-1.0 True
1.0 False
1.0 False
1.0 False
-1.0 True
1.0 False
1.0 False
1.0 False
